In [51]:
import pandas as pd

In [52]:
df = pd.read_csv("C:\\Users\\thoma\\Desktop\\ev-modelling-repo\\Data\\NHTS_trips_raw.csv")

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923572 entries, 0 to 923571
Columns: 105 entries, HOUSEID to HBRESDN
dtypes: float64(4), int64(97), object(4)
memory usage: 739.9+ MB


In [54]:
data = pd.DataFrame()

# Übernehmen relevanter Spalten und Umbennennung nach MiD Variablen

In [55]:
# zusammenfügen von PERSONID und HOUSEID um einzigartige personenbezogene ID zu erlangen
data["ID"] = df["HOUSEID"].apply(lambda x: str(x)) + df["PERSONID"].apply(lambda x: str(x)) 
data["ID"] = data["ID"].apply(lambda x: int(x))
data["ST_WOTAG"] = df["TRAVDAY"]
data["WHYFROM"] = df["WHYFROM"]
data["zweck"] = df["WHYTO"]
data["W_NR"] = df["TDTRPNUM"]
data["TRPTRANS"] = df["TRPTRANS"]
data["wegkm"] = df["VMT_MILE"]
data["wegmin_imp1"] = df["TRVLCMIN"]
data["W_SZ"] = df["STRTTIME"]
data["W_AZ"]= df["ENDTIME"]

In [56]:
testing = data.copy()

In [57]:
testing.head()

,ID,ST_WOTAG,WHYFROM,zweck,W_NR,TRPTRANS,wegkm,wegmin_imp1,W_SZ,W_AZ
0,300000071,2,1,19,1,3,5.244,15,1000,1015
1,300000071,2,19,1,2,3,5.149,20,1510,1530
2,300000072,2,3,1,1,6,84.004,120,700,900
3,300000072,2,1,3,2,6,81.628,150,1800,2030
4,300000073,2,1,8,1,3,2.250,15,845,900


# Zuordnung der Wochentage anpassen

In [58]:
testing["ST_WOTAG"] = testing["ST_WOTAG"].replace({1:7, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6})

In [59]:
testing["ST_WOTAG"].value_counts()

5    154049
4    150812
3    147798
2    147502
1    137379
6     97536
7     88496
Name: ST_WOTAG, dtype: int64

In [60]:
data["ST_WOTAG"].value_counts()

6    154049
5    150812
4    147798
3    147502
2    137379
7     97536
1     88496
Name: ST_WOTAG, dtype: int64

# Umrechnen der Wergzeiten von hhmm in Minuten

In [61]:
testing["W_SZ"]

0         1000
1         1510
2          700
3         1800
4          845
          ... 
923567     810
923568    1320
923569    1415
923570    1820
923571    1848
Name: W_SZ, Length: 923572, dtype: int64

In [62]:
def to_minutes(hhmm):
    result = 0
    if len(str(hhmm)) == 4:
        hhmm = str(hhmm)
        hh = int(hhmm[0:2])
        mm = int(hhmm[2:4])
    if len(str(hhmm)) ==3:
        hmm = str(hhmm)
        hh = int(hmm[0])
        mm = int(hmm[1:3])
    if len(str(hhmm)) ==2:
        mm = str(hhmm)
        hh = 0
        mm = int(mm)
    if len(str(hhmm)) ==1:
        m = str(hhmm)
        hh = 0
        mm = int(m)
    result = hh*60+mm
    return result

In [63]:
x = 14*60+50
test = to_minutes(1450)
x==test

True

In [64]:
testing["W_SZ_MIN"] = testing["W_SZ"].apply(lambda x: to_minutes(x))

In [65]:
testing["W_AZ_MIN"] = testing["W_AZ"].apply(lambda x: to_minutes(x))

In [66]:
testing.head(10)

,ID,ST_WOTAG,WHYFROM,zweck,W_NR,TRPTRANS,wegkm,wegmin_imp1,W_SZ,W_AZ,W_SZ_MIN,W_AZ_MIN
0,300000071,1,1,19,1,3,5.244,15,1000,1015,600,615
1,300000071,1,19,1,2,3,5.149,20,1510,1530,910,930
2,300000072,1,3,1,1,6,84.004,120,700,900,420,540
3,300000072,1,1,3,2,6,81.628,150,1800,2030,1080,1230
4,300000073,1,1,8,1,3,2.250,15,845,900,525,540
5,300000073,1,8,1,2,3,2.243,15,1430,1445,870,885
6,300000081,4,1,3,1,6,8.017,15,1115,1130,675,690
7,300000081,4,3,1,2,6,8.017,10,2330,2340,1410,1420
8,300000121,4,1,16,1,4,3.395,15,550,605,350,365
9,300000121,4,16,1,2,4,3.395,15,700,715,420,435


# Neuzuordnung Wegzwecke
-7 (I prefer not to answer) -> 99 (keine Angabe) \
-8 (I don't know) -> 99 (keine Angabe) \
-9 (Not ascertained) -> 99 (keine Angabe) \
01 (Regular home activities (chores, sleep)) -> 8 (nach Hause) \
02 (Work from home(paid)) -> 8 (nach Hause) \
03 (Work) -> 1 (Erreichen des Arbeitsplatzes) \
04 (Work-related meeting / trip) -> 2 (dienstlich/geschäftlich) \
05 (Volunteer acitivities (not paid)) -> 7 (Freizeitaktivität) \
06 (Drop off / pick up someone) -> 6 (Bringen/Holen/Begleiten von Personen) \
07 (Change type of transportation) -> 10 (anderer Zweck) \
08 (Attend school as a student) -> 3 (Erreichen der Ausbildungsstätte/Schule) \
09 Attend child care -> 10 (anderer Zweck) \
10 Attend adult care -> 10 (anderer Zweck) \
11 Buy goods (groceries, clothes, appliances, gas) -> 4 (Einkauf) \
12 Buy services (dry cleaners, banking, service a car, pet care) -> 5 (private Erledigungen) \
13 Buy meals (go out for a meal, snack, carry-out) -> 7 (Freizeitaktivität) \
14 Other general errands (post office, library) -> 5 (private Erledigungen) \
15 Recreational activities (visit parks, movies, bars, museums) -> 7 (Freizeitaktivität) \
16 Exercise (go for a jog, walk, walk the dog, go to the gym) -> 7 (Freizeitaktivität) \
17 Visit friends or relatives -> 7 (Freizeitaktivität) \
18 Health care visit (medical, dental, therapy) -> 5 (private Erledigungen) \
19 Religious or other community activities	-> 7 (Freizeitaktivität) \
97 Something Else -> 10 (anderer Zweck)

## Achtung: 
### Im Ausgangsdatensatz 9 = Rückweg vom vorherigen Weg, kommt hier nicht zum Einsatz

In [67]:
reassignment = {-7:99, -8:99, -9:99, 1:8, 2:8, 3:1, 4:2, 5:7, 6:6, 7:10, 8:3, 9:10, 10:10, 11:4,
                12:5, 13:7, 14:5, 15:7, 16:7, 17:7, 18:5, 19:7, 97:10}

In [68]:
testing["zweck"] = testing["zweck"].replace(reassignment)
testing["WHYFROM"] = testing["WHYFROM"].replace(reassignment)

In [69]:
testing["zweck"].value_counts()

8     318777
7     199666
4     132545
1      98968
5      68365
6      56377
3      22133
10     14887
2      11622
99       232
Name: zweck, dtype: int64

# Reduzieren der Zustände
auf: 
* Zuhause -> 1
* Arbeit -> 2 
* Einkaufen -> 3 
* Freizeit -> 4
* Sonstiges -> 5

### Zuordnung:
* Erreichen des Arbeitsplatzes 1 -> 2
* dienstlich/geschäftlich 2 -> 5  
* Erreichen der Ausbildungsstätte/Schule 3 -> 2 
* Einkauf (Dienstleistungen) 4 -> 3
* private Erledigung 5 -> 3
* Bringen/Holen/Begleiten von Personen 6 -> 5
* Freizeitaktivität 7 -> 4 
* nach Hause 8 -> 1
* Rückweg vom vorherigen Weg 9 -> zu letztem Zustand (in NHTS Datensatz nicht vorhanden)
* anderer Zweck 10 -> 5 Sonstiges
* keine Angabe 99 -> 5 Sonstiges (oder nicht berücksichtigen?)

In [70]:
reduzieren = {1:2, 2:5, 3:2, 4:3, 5:3, 6:5, 7:4, 8:1, 10:5, 99:5}

In [71]:
testing["zweck"] = testing["zweck"].replace(reduzieren)
testing["WHYFROM"] = testing["WHYFROM"].replace(reduzieren)

# Umrechnen Wegelänge

1mile = 1.60934km

In [72]:
testing["wegkm"] = testing["wegkm"].apply(lambda x: round(x*1.60934, 3) if x > 0 else -1)

In [73]:
testing["wegkm"]

0           8.439
1           8.286
2         135.191
3         131.367
4           3.621
           ...   
923567     -1.000
923568     -1.000
923569     -1.000
923570     -1.000
923571     -1.000
Name: wegkm, Length: 923572, dtype: float64

In [74]:
testing.head(10)

,ID,ST_WOTAG,WHYFROM,zweck,W_NR,TRPTRANS,wegkm,wegmin_imp1,W_SZ,W_AZ,W_SZ_MIN,W_AZ_MIN
0,300000071,1,1,4,1,3,8.439,15,1000,1015,600,615
1,300000071,1,4,1,2,3,8.286,20,1510,1530,910,930
2,300000072,1,2,1,1,6,135.191,120,700,900,420,540
3,300000072,1,1,2,2,6,131.367,150,1800,2030,1080,1230
4,300000073,1,1,2,1,3,3.621,15,845,900,525,540
5,300000073,1,2,1,2,3,3.610,15,1430,1445,870,885
6,300000081,4,1,2,1,6,12.902,15,1115,1130,675,690
7,300000081,4,2,1,2,6,12.902,10,2330,2340,1410,1420
8,300000121,4,1,4,1,4,5.464,15,550,605,350,365
9,300000121,4,4,1,2,4,5.464,15,700,715,420,435


## Zuordnen Satrtzeiten zu Zeitschritten
Schrittweite: t = 15 Minuten \
96 Zeitschritte pro Tag: W_SZ_t = [0, 96]

In [75]:
testing["W_SZ_t"] = testing["W_SZ_MIN"].apply(lambda x: round(x/15))

In [76]:
testing.head()

,ID,ST_WOTAG,WHYFROM,zweck,W_NR,TRPTRANS,wegkm,wegmin_imp1,W_SZ,W_AZ,W_SZ_MIN,W_AZ_MIN,W_SZ_t
0,300000071,1,1,4,1,3,8.439,15,1000,1015,600,615,40
1,300000071,1,4,1,2,3,8.286,20,1510,1530,910,930,61
2,300000072,1,2,1,1,6,135.191,120,700,900,420,540,28
3,300000072,1,1,2,2,6,131.367,150,1800,2030,1080,1230,72
4,300000073,1,1,2,1,3,3.621,15,845,900,525,540,35


## Speichern der Daten

In [77]:
testing.to_csv(r'C:\Users\thoma\Desktop\ev-modelling-repo\Data\NHTS_trips_processed.csv', header=True, index=False) 